In [56]:
from bs4 import BeautifulSoup
import pandas as pd

In [57]:
# Read in the bookmarks file
with open('bookmarks_18_01_2024.html', 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')

In [58]:
# Add a dummy h3 because we capture between each pair of h3 tags later
soup.append(BeautifulSoup('<h3>dummy</h3>', 'html.parser'))

In [59]:
# Each H3 tag is a folder, each A tag is a bookmark
# They are not wrapped in tags so that's pretty fucking annoying
# So first find all H3 tags
h3s = soup.find_all("h3")

In [60]:
import re
# Now we can find the content between each pair of h3 tags
# We'll make a regex pattern and match everything between them and save to a dict
# For the last h3 we'll just match everything after
contents = {}

for i in range(0, len(h3s) - 1):
    pattern = re.compile(r'{}(.+?){}'.format(re.escape(str(h3s[i])), re.escape(str(h3s[i + 1]))), re.DOTALL)
    heading = h3s[i].text
    try:
        contents[heading] = pattern.search(str(soup)).group(1).strip()
    except AttributeError:
        print(f"Failed for heading {heading}")


In [62]:
contents

{'Bookmarks bar': '<dl><p>\n</p></dl><p>\n<dt>',
 'Sewing Patterns': '<dl><p>\n<dt><a add_date="1460295122" href="https://www.youtube.com/watch?v=TW9t0HTK_7E&amp;feature=youtu.be" icon="">Cosmetics bag</a>\n<dt><a add_date="1460295147" href="http://verypurpleperson.com/2013/01/tutorial-sewing-panties/" icon="">Underwear</a>\n<dt><a add_date="1460309386" href="http://www.instructables.com/id/Bra/" icon="">Bra</a>\n<dt><a add_date="1460400757" href="http://www.measuretwicecutonce.com.au/product/the-darcy-boxer-shorts/" icon="">Boxer Shorts</a>\n<dt><a add_date="1460837564" href="http://m-sewing.com/" icon="">Modern Sewing Patterns</a>\n<dt><a add_date="1674939958" href="https://web.archive.org/web/20200217005037/http://www.craftstylish.com:80/item/64549/make-a-one-hour-dress/page/all" icon="">Make a One-Hour Dress - CraftStylish</a>\n<dt><a add_date="1675590870" href="https://blog.spoonflower.com/2019/09/17/diy-fanny-pack-free-pattern/" icon="">Make Your Own Fanny Pack with This Free Pat

In [63]:
# Parse the dictionary and pop it into a pandas dataframe
df = pd.DataFrame()

for key, value in contents.items():
    # Make the value a soup object again
    value = BeautifulSoup(value, 'html.parser')
    bookmarks = value.find_all('a')
    for bookmark in bookmarks:
        title = bookmark.text
        url = bookmark.get('href')
        add_date = bookmark.get('add_date')
        df = df.append({'folder': key, 'title': title, 'url': url, 'add_date': add_date,}, ignore_index=True)

/tmp/ipykernel_21704/3671549670.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'folder': key, 'title': title, 'url': url, 'add_date': add_date,}, ignore_index=True)
/tmp/ipykernel_21704/3671549670.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'folder': key, 'title': title, 'url': url, 'add_date': add_date,}, ignore_index=True)
/tmp/ipykernel_21704/3671549670.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'folder': key, 'title': title, 'url': url, 'add_date': add_date,}, ignore_index=True)
/tmp/ipykernel_21704/3671549670.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [64]:
df.head()

,folder,title,url,add_date
0,Sewing Patterns,Cosmetics bag,https://www.youtube.com/watch?v=TW9t0HTK_7E&fe...,1460295122
1,Sewing Patterns,Underwear,http://verypurpleperson.com/2013/01/tutorial-s...,1460295147
2,Sewing Patterns,Bra,http://www.instructables.com/id/Bra/,1460309386
3,Sewing Patterns,Boxer Shorts,http://www.measuretwicecutonce.com.au/product/...,1460400757
4,Sewing Patterns,Modern Sewing Patterns,http://m-sewing.com/,1460837564


In [65]:
# Convert add date to datetime
df['add_date'] = pd.to_datetime(df['add_date'], unit='s')

In [66]:
df.head()

,folder,title,url,add_date
0,Sewing Patterns,Cosmetics bag,https://www.youtube.com/watch?v=TW9t0HTK_7E&fe...,2016-04-10 13:32:02
1,Sewing Patterns,Underwear,http://verypurpleperson.com/2013/01/tutorial-s...,2016-04-10 13:32:27
2,Sewing Patterns,Bra,http://www.instructables.com/id/Bra/,2016-04-10 17:29:46
3,Sewing Patterns,Boxer Shorts,http://www.measuretwicecutonce.com.au/product/...,2016-04-11 18:52:37
4,Sewing Patterns,Modern Sewing Patterns,http://m-sewing.com/,2016-04-16 20:12:44


In [68]:
df.shape

(135, 4)

In [69]:
df.to_csv('bookmarks.csv', index=False)